<a href="https://colab.research.google.com/github/pavanreddy565/Tensor_flow-learning/blob/main/Tutorials/Hyper_parameter_Tunning_using_Keras_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install keras-tuner --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.9 MB/s eta 0:00:00


In [6]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner import RandomSearch

In [8]:
df = pd.read_csv('Real_Combine.csv')
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [9]:
X = df.iloc[:,:-1] ## independent features
y = df.iloc[:,:-1] ## dependent features

### Hyperparameters
1. How many number of hidden layers we should have?
2. How many neurons we should have in hidden layers?
3. Learning Rate

In [10]:
def bulid_model(hp):
  model = keras.Sequential()
  for i in range(hp.Int('num_layers',2,20)):
    model.add(layers.Dense(units=hp.Int('units_'+str(i),
                                        min_value=32,
                                        max_value=512,
                                        step=32),
                           activation="relu"))
  model.add(layers.Dense(1,activation="linear"))
  model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
        ), loss="mean_absolute_error",
        metrics=['mean_absolute_error']
  )
  return model

In [15]:
tuner = RandomSearch(
    bulid_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='project',
    project_name='Air Quality Index'
)

In [16]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}


In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=3)

In [20]:
tuner.search(X_train, y_train, epochs=5,
             validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 16s]
val_mean_absolute_error: 136.6528523763021

Best val_mean_absolute_error So Far: 136.6528523763021
Total elapsed time: 00h 01m 57s


In [21]:
tuner.results_summary()

Results summary
Results in project/Air Quality Index
Showing 10 best trials
Objective(name="val_mean_absolute_error", direction="min")

Trial 4 summary
Hyperparameters:
num_layers: 4
units_0: 512
learning_rate: 0.001
units_1: 64
units_2: 448
units_3: 384
units_4: 512
units_5: 128
units_6: 224
units_7: 96
units_8: 64
units_9: 96
units_10: 384
units_11: 384
units_12: 96
units_13: 352
units_14: 256
units_15: 288
units_16: 448
units_17: 160
Score: 136.6528523763021

Trial 0 summary
Hyperparameters:
num_layers: 18
units_0: 480
learning_rate: 0.01
units_1: 224
units_2: 32
units_3: 32
units_4: 32
units_5: 32
units_6: 32
units_7: 32
units_8: 32
units_9: 32
units_10: 32
units_11: 32
units_12: 32
units_13: 32
units_14: 32
units_15: 32
units_16: 32
units_17: 32
Score: 136.84221394856772

Trial 3 summary
Hyperparameters:
num_layers: 7
units_0: 160
learning_rate: 0.0001
units_1: 224
units_2: 416
units_3: 192
units_4: 288
units_5: 384
units_6: 128
units_7: 288
units_8: 320
units_9: 32
units_10: 384
